In [1]:
import numpy as np
import pandas as pd
import math


In [2]:
df=pd.read_csv("data3_19.csv")

In [3]:
print(df)

     pclass    age  gender survived
0       1st  adult    male      yes
1       1st  adult    male      yes
2       1st  adult    male      yes
3       1st  adult    male      yes
4       1st  adult    male      yes
5       1st  adult    male      yes
6       1st  adult    male      yes
7       1st  adult    male      yes
8       1st  adult    male      yes
9       1st  adult    male      yes
10      1st  adult    male      yes
11      1st  adult    male      yes
12      1st  adult    male      yes
13      1st  adult    male      yes
14      1st  adult    male      yes
15      1st  adult    male      yes
16      1st  adult    male      yes
17      1st  adult    male      yes
18      1st  adult    male      yes
19      1st  adult    male      yes
20      1st  adult    male      yes
21      1st  adult    male      yes
22      1st  adult    male      yes
23      1st  adult    male      yes
24      1st  adult    male      yes
25      1st  adult    male      yes
26      1st  adult    male  

In [4]:
class Node:
    def __init__(self):
        self.attr=""
        self.class_val=""
        self.child_nodes=[]
    
    

In [5]:
r=[1,1000,2149]
print(df.iloc[r])

     pclass    age  gender survived
1       1st  adult    male      yes
1000    3rd  adult    male       no
2149   crew  adult  female       no


In [6]:
def log2(x):
    return math.log(x)/math.log(2)
def pos_neg_count(df):
    num_rows=len(df.axes[0])
    num_cols=len(df.axes[1])-1#neglecting the class column
    
    pos_df=df[df["survived"]=="yes"]
    pos_count=len(pos_df.axes[0])
    neg_count=num_rows-pos_count
    return pos_count,neg_count
def entropy(pos,neg,num_rows):
    if(pos==0 or neg==0):
        return 0
    pos_pr=pos/num_rows
    neg_pr=neg/num_rows
    E=-pos_pr*log2(pos_pr)-neg_pr*log2(neg_pr)
    return E

In [7]:

def form_tree(df,nd,attr_list,level,print_flag):
   
    #print("ATTR LIST = "+str(attr_list))
    #df = dataframe
    #nd=Current Node
    
    #print(df["survived"].unique()) //Two classes = "yes" and "no"
    num_rows=len(df.axes[0])
    num_cols=len(df.axes[1])-1#neglecting the class column
    pos,neg=pos_neg_count(df)
    
    t="\t"*level
    if(pos==0):
        nd.class_val="no"
        if(print_flag==1): print(t+"##Class = no")
        
        return nd
    if(neg==0):
        nd.class_val="yes"
        if(print_flag==1): print(t+"##Class = yes")
        return nd
    
    
    #Finding Prior Entropy
    E1=entropy(pos,neg,num_rows)
    #print(E1)
    #E1==prior Entropy
    
    max_attr=""
    max_diff=-99999
    #Finding Posterior Entropy
    for c in attr_list:
        u=list(df[c].unique())
        if(len(u)==1):
            attr_list.remove(c)
            continue
        else:
            E2=0
            for val in u:
                dfv=df[df[c]==val]
                num_rowsv=len(dfv.axes[0])
                posv,negv=pos_neg_count(dfv)
                E=entropy(posv,negv,num_rowsv)
                E2=E2+num_rowsv*1.0*E/num_rows
            Ediff=E1-E2
            if(Ediff>max_diff):
                max_diff=Ediff
                max_attr=c
                
    
    if(max_diff!=-99999):
        #If a attribute is Chosen, we create child nodes and recursively call function on them
        nd.attr=max_attr
        attr_list.remove(nd.attr)
        u=list(df[max_attr].unique())
        for val in u:
            df1=df[df[max_attr]==val]
            
            if(print_flag==1): print(t+"##"+max_attr+" = "+str(val))
            RN=Node()
            nd.child_nodes.append([val,RN])
            form_tree(df1,RN,attr_list,level+1,print_flag)
        attr_list.append(nd.attr)
        return nd
    else:
        #If no more attribute to choose, then we assign class as majority class
        if(pos>=neg):
            nd.class_val="yes"
            if(print_flag==1): print(t+"##Class = yes")
            return nd
        else:
            nd.class_val="no"
            if(print_flag==1): print(t+"##Class = no")
            return nd
            
        
            
            
    
            
            
                
    

In [8]:
def form_dec_tree(df,print_flag):
    columns=list(df.columns)
    attr_list=columns[0:len(columns)-1] # attr_list = possible attributes to choose from
    RN=Node() # Node created for root node
    RN=form_tree(df,RN,attr_list,0,print_flag)
    return RN
def predict(df,row,tree):
    node=tree
    #print(r)
    #print("\n\n")
    while(node.class_val==""):
        attr=node.attr
        val=df.at[row,attr]
        for l in node.child_nodes:
            if(l[0]==val):
                node=l[1]
    return node.class_val
    
    

    
    
#tree=form_dec_tree(df,1)
#print(list(df.iloc[1]))
#predict(df,1,tree)

In [11]:
def AdaBoost_train(df):
    num_rows=len(df.axes[0])
    classifiers=[] # will be of the form classifiers=[[Tree1,Alpha1],[Tree2,Alpha2],[Tree3,Alpha3]]
    weight_rows=np.ones(num_rows)
    weight_rows=weight_rows/num_rows
    for i in range(0,3):
        Tree=form_dec_tree(df,1)
        #print(df.iloc[0])
        target=[[predict(df,int(row),Tree),df.at[row,"survived"]] for row in df.index]
        #each element of target = [predicted class,actual class]
        #print(target)
        eq=[(target[r][0]==target[r][1])*1 for r in range(0,num_rows)]
        acc=sum(eq)*100.0/num_rows
        print("Classifier "+str(i+1)+" trained with accuracy "+str(acc))
        rows=[index for index,value in enumerate(eq) if value==0] # Misclassified rows
        mis_weights=[weight_rows[r] for r in rows] # Weights of misclassified rows
        error_rate=sum(mis_weights)*1.0/num_rows
        alpha=0.5*math.log((1-error_rate)/(error_rate*1.0))
        print("Classifier "+str(i+1)+" has error rate,alpha = "+str(error_rate)+","+str(alpha))
        for i in range(0,num_rows):
            if(eq[i]==1):#Correctly Classified
                weight_rows[i]=weight_rows[i]*math.exp(alpha)
        print(np.unique(weight_rows))
        weight_rows=weight_rows*1.0/np.sum(weight_rows)
        #print(pos_neg_count(df))
        df=df.sample(num_rows,replace=True,axis=0,weights=weight_rows)
        df=df.reset_index(drop=True)
        #print(df)
       
        #print(pos_neg_count(df))
        #print(df)
     
        
        
    #print(weight_rows)
    
    
    
    return classifiers

AdaBoost_train(df)

##gender = male
	##pclass = 1st
		##age = adult
			##Class = no
		##age = child
			##Class = yes
	##pclass = 2nd
		##age = adult
			##Class = no
		##age = child
			##Class = yes
	##pclass = 3rd
		##age = adult
			##Class = no
		##age = child
			##Class = no
	##pclass = crew
		##Class = no
##gender = female
	##pclass = 1st
		##Class = yes
	##pclass = 2nd
		##Class = yes
	##pclass = 3rd
		##Class = no
	##pclass = crew
		##Class = yes
Classifier 1 trained with accuracy 79.30232558139535
Classifier 1 has error rate,alpha = 9.62682531097882e-05,4.624137843594995
[0.00046512 0.04740226]
##gender = male
	##age = adult
		##pclass = crew
			##Class = no
		##pclass = 3rd
			##Class = no
		##pclass = 2nd
			##Class = no
		##pclass = 1st
			##Class = no
	##age = child
		##pclass = 3rd
			##Class = no
		##pclass = 2nd
			##Class = yes
		##pclass = 1st
			##Class = yes
##gender = female
	##pclass = 1st
		##Class = yes
	##pclass = 3rd
		##Class = no
	##pclass = 2nd
		##Class = yes
	##pclass = crew
		

ZeroDivisionError: float division by zero

In [ ]:
df1=df[(df["gender"]=="male") & (df["pclass"]=="2nd") & (df["age"]=="adult")]
pos,neg=pos_neg_count(df1)
print("Yes "+str(pos))
print("No "+str(neg))